In [8]:
import regex as re
try:
    from services.sapling import get_sapling_edits 
except:
    from sapling import get_sapling_edits

In [9]:
def sapling_to_gingerit_format(sapling_response, original_text):
    print(f"original text:  {original_text} {len(original_text)}")
    corrected_text = original_text

    corrections = []
    offset = 0  # Initialize an offset to track cumulative position shifts

    for correction in sapling_response:
        start = correction['start'] + offset  # Adjust start position based on cumulative offset
        end = correction['end'] + offset  # Adjust end position based on cumulative offset
        replacement = correction['replacement']
        general_error_type = correction['general_error_type']

        # Calculate the length difference between the replaced text and the replacement
        length_diff = len(replacement) - (end - start)

        # Apply the correction to the text
        corrected_text = corrected_text[:start] + replacement + corrected_text[end:]
        print(f"""after correction :  {corrected_text} \n
              length of word to be replaced: {end - start} \n
              length of replacement: {len(replacement)} \n
              corrected result: {corrected_text}""")

        # Update the offset based on the length difference
        offset += length_diff

        # Create the correction object
        correction_obj = {
            "start": start,
            "text": original_text[start:end],
            "correct": replacement,
            "definition": general_error_type
        }

        corrections.append(correction_obj)

    # Create the desired output format
    output = {
        "text": original_text,
        "result": corrected_text,  # Use the corrected_text here
        "corrections": corrections
    }

    return output


In [10]:
def remove_emoticons_hashtags_tags(text):
    # Remove emoticons
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', '', text)

    # Remove hashtags and @tags
    text +=" "
    text = re.sub(r'[#@]\S+\s', '', text)

    # Remove anything inside parentheses
    text = re.sub(r'\([^)]*\)', '', text)
    
    # Remove any trailing punctuations
    text = text.replace("@","").replace("#","")
    
    return text.strip()

In [14]:
original_text = """This #FriendshipDay, check out how we worked with our clients to celebrate the people who make our days a little brighter. 👩🏽‍🤝‍👩🏻💕"""
emoticonsless_text=remove_emoticons_hashtags_tags(original_text)
emoticonsless_text

'This check out how we worked with our clients to celebrate the people who make our days a little brighter. \u200d\u200d'

In [17]:
get_sapling_edits(emoticonsless_text)

"Error: {'key_prefix': 'NHU********************************', 'msg': 'Rate Limited. Visit https://sapling.ai/docs/api/api-access for details.'}"

In [ ]:
gingerit_output = sapling_to_gingerit_format(get_sapling_edits(emoticonsless_text), emoticonsless_text)
print(gingerit_output)